## 1.Pre-Processing Data Before Import ( Convert Into Suitable Dataset )

### Movie DataSet

In [268]:
# removed_movie = []
# index = 0
# for movie in tqdm(np_movies):
#     if movie[0] not in np_genome[:,0]:
#         np_movies = np.delete(np_movies, index,0)
#         removed_movie.append(index)
#         continue
#     index += 1

100%|███████████████████████████████████████████████████████████████████████████| 62423/62423 [04:35<00:00, 226.97it/s]


In [350]:
# processed_movies = pd.DataFrame(np_movies, columns=movies.columns)
# processed_movies.to_csv('processed_movies.csv', encoding='utf-8', index=False)

### Mostly Genome ( Scores + Tags )

In [326]:
# def preprocess_tag():    
#     sort_tags = sorted(tag_count.items(), key=lambda x:x[1], reverse=True)[:70]
#     mostly_tags_df = pd.DataFrame(columns=genome_tags.columns)
#     tags_id = {}
#     for tag in sort_tags:
#         count = genome_tags.tag == tag[0]
#         if count.sum() != 1:
#             continue
#         else:
#             tag_index = genome_tags.tagId[count].values[0]
#             concat = pd.DataFrame([[tag_index, tag[0]]], columns=genome_tags.columns)
#             tags_id[tag[0]] = tag_index
#             mostly_tags_df = pd.concat([mostly_tags_df,concat], ignore_index=True)
#     return tags_id, mostly_tags_df

# tags_id, mostly_tags_df = preprocess_tag()

# genome = pd.merge(genome_scores,mostly_tags_df,on='tagId')



In [330]:
# genome.to_csv('processed_genome1.csv', encoding='utf-8', index=False)

### Tags Count

In [ ]:
# tag_count = {}
# for i in tqdm(range(len(np_genome_tags))):
#     count =  tags.tag[tags.tag == i].count()
# #     if count > 10:
#     tag_count[i] = int(count)

# import json
# with open("Tag_Count.json", "w") as write_file:
#     json.dump(tag_count, write_file, indent=4)

## 2. Import Libraries

In [1]:
import pandas as pd 
import numpy as np
from tqdm import tqdm
import json

## 3. Import DataSet and Convert into Array

In [331]:
# ratings = pd.read_csv("ratings.csv")
# movies = pd.read_csv("movies.csv")
# tags = pd.read_csv("tags.csv")
# genome_scores = pd.read_csv("genome-scores.csv")
# genome_tags = pd.read_csv("genome-tags.csv")
# genome = pd.merge(genome_scores,genome_tags,on='tagId')

In [4]:
ratings = pd.read_csv("ratings.csv")
movies = pd.read_csv("processed_movies.csv")
tags = pd.read_csv("tags.csv")
genome = pd.read_csv("processed_genome.csv")

In [5]:
# Opening JSON file
f = open('Tag_Count.json')
  
# returns JSON object as a dictionary
tag_count = json.load(f)

In [6]:
np_genome = genome.to_numpy()
np_movies = movies.to_numpy()
np_tags = tags.to_numpy()
np_ratings = ratings.to_numpy()
# np_genome_tags = genome_tags.to_numpy()

## 4. Requirement Dict

### 4.1 Tags_Id

In [7]:
tags_id = {}
for index, tag in enumerate(np.unique(genome.tag)):
    tags_id[tag] = (np.unique(genome.tagId)[index],index)

### 4.2 Movies_Id

In [8]:
movies_id = {}
for index, movie in enumerate(np_movies):
    movies_id[movie[1]] = (movie[0], index)

In [9]:
item = 'Shrek the Third (2007)'
retrieved_item = 'Shrek (2001)'
tag = 'action'
direction = 0.5

## 5.2.1. Critique Distance

In [10]:
def rel_item(item,tag):
    movie_id = movies_id[item][0]
    tag_id = tags_id[tag][0]
    return float(np_genome[(np.where((np_genome[:,0] == movie_id) & (np_genome[:,1] == tag_id)))][:,2])

In [14]:
rel_all_items(tag)

0.2554999999999999

In [11]:
#rel_item_optimized 
def rel_all_items(tag):
    tag_index = tags_id[tag][1]
    len_movie = len(movies)
    rel_items = np_genome[tag_index*len_movie:tag_index*len_movie+len_movie,2].tolist()
    return rel_items

In [12]:
def linear_sat(item,tag,direction):
    a = (rel_all_items(tag) - np.array(rel_item(item,tag)))*direction
    result = np.clip(a, 0, 1)
    return result

In [13]:
def diminish_sat(item,tag,direction):
    critique_dist = linear_sat(item,tag,direction)
    result = 1 - np.exp(-5*critique_dist)
    return result

In [15]:
rel_item(retrieved_item,tag)

0.4725

In [16]:
rel_item(item,tag)

0.2554999999999999

In [17]:
result = diminish_sat(item,tag,direction)
result

array([0.63850244, 0.61782437, 0.        , ..., 0.07109611, 0.01673341,
       0.44289414])

## 5.2.2. Item Similarity

In [18]:
def population():
    populations = {}
    for tag in tags_id:
        if tag not in tag_count.keys():
            tag_count[tag] = 0
        populations[tag] = tag_count[tag]
    return populations

In [19]:
populations = population()

In [20]:
def single_doc_freq(tag):
    # used to compute as the number of items where relevant greater tha 0.5 for only one tag 
    index = np.where(np_genome[:,3] == tag)
    count = (np_genome[index][:,2] > 0.5).sum()
    return count

In [21]:
single_doc_freq('action')

2139

In [22]:
def full_doc_freq():
    #used to comute doc_freq for all tags
    doc_freqs = {tag:0 for tag in tags_id}
    len_movies = len(movies)
    for i in tqdm(range(0,len(np_genome),len_movies)):
        tag = np_genome[i][-1]
        doc_freqs[tag] = (np_genome[i:i+len_movies,2] > 0.5).sum()
    return doc_freqs

In [23]:
doc_freqs = full_doc_freq()

100%|████████████████████████████████████████████████████████████████████████████████| 65/65 [00:00<00:00, 2496.02it/s]


In [24]:
def tags_weight(populations, doc_freqs, alpha = 0.000001):
    doc_freqs = np.array(list(doc_freqs.values()))
    populations = np.array(list(populations.values()))
    result = (np.array(populations) + alpha) / (doc_freqs + alpha)
    return np.clip(result,0,result.max())

In [25]:
weights = tags_weight(populations, doc_freqs)
weights

array([ 2.76157083,  2.30887097,  8.61682241,  3.59594594, 25.0845067 ,
        3.66685425,  2.73800539,  3.72580645,  1.75601375,  5.03260869,
       61.48275654,  0.91064757,  2.75159236,  2.21867704,  3.2527881 ,
        1.53343239,  1.65985915,  2.20468278,  9.72463766,  2.34830508,
        0.63916769,  3.06050955,  4.01569858, 13.17956653,  0.77253548,
        3.98529411,  1.38566351,  2.84223155,  7.01362396,  3.40687161,
        2.14438944,  0.7780154 ,  0.72556657,  1.19333333,  2.93388429,
        6.78694156,  1.3875803 ,  3.681203  ,  4.12676056,  2.86498516,
       10.69483564,  0.66881973,  1.41977501,  3.30446673,  3.12404418,
        1.55396066,  2.81649331,  3.52139461, 10.55766792,  3.06083086,
        1.12689461,  8.10644256,  1.88950836, 10.89591074,  4.73001776,
        0.8964066 ,  0.81458824,  4.53301886,  2.39583333, 24.79389295,
        3.20495185,  5.78631452,  1.32269645,  1.3909035 ,  2.84722222])

In [26]:
def tag_item_vector(item):
    item_id = movies_id[item][0]
    full_tags_item = [np_genome[i][2] for i in np.where(np_genome[:,0] == item_id)[0]]  
#     if np.shape(full_tags_item) != (1128,):
#         full_tags_item = (np.zeros((1128,))+0.5).tolist()
    return full_tags_item

In [35]:
def cos_similarity(item, weights):
    
    # Create all movies's tags scores
    list_tags = np.zeros((len(movies),len(tags_id)))

    for index,tag_score in enumerate(np_genome[:,2]):
        tags_index = index // len(movies)
        movies_index = index - tags_index*len(movies)
        list_tags[movies_index,tags_index] = tag_score
        
    # Extract 1 query movie from matrix above 
    query_index = movies_id[item][1]
    query_tags = list_tags[query_index]
    
    numerator = np.dot(list_tags * list_tags[query_index],weights)
    
    denominator = np.sqrt(np.dot(list_tags**2, weights)) * np.sqrt(np.dot(query_tags**2, weights))
    
    result = numerator / denominator
    return result

In [36]:
cos_similarity('Shrek the Third (2007)',weights)

array([0.72398614, 0.5960863 , 0.79903563, ..., 0.54558482, 0.60957284,
       0.62228295])

In [235]:
def encode_movies():
    genres = {}
    index = 0
    movies_encode = []
    for movie in np_movies[:,2]:
        movie_encode = [0]*20
        for genre in movie.split('|'):
            if genre not in genres:
                genres[genre] = index
                index += 1
            movie_encode[genres[genre]] = 1
        movies_encode.append(movie_encode)
    return movies_encode

In [236]:
movies_encode = encode_movies()

In [296]:
def genres_similarity(item):
    index = movies_id[item][1]
    numerator = np.dot(movies_encode, movies_encode[index])
    denominator = np.sqrt(np.square(movies_encode[index]).sum()) * np.sqrt(np.sum(np.square(movies_encode),axis=1))
    return numerator/denominator

In [297]:
genres_similarity('Toy Story (1995)')

array([1.        , 0.77459667, 0.31622777, ..., 0.        , 0.4472136 ,
       0.4472136 ])

In [403]:
item = 'Shrek the Third (2007)'
retrieved_item = 'Shrek (2001)'
tag = 'classic'
direction = 0.5

In [404]:
def recommendation(item, tag, direction):    
    value = diminish_sat(item,tag,direction)
    cos = cos_similarity(item, weights)
    genre_simi = genres_similarity(item)

    a = value * cos * genre_simi
    # progress_bar = tqdm(np_movies[:,1])

    score = {}

    for index, movie in enumerate(np_movies[:,1]):
        score[movie] = a[index]
    
    result = sorted(score.items(), key=lambda x:x[1], reverse=True)[:10]
    return result

### Toi day thoi nha, Phan sau t de lai update dan` dan` a' 

In [417]:
movies_id["Twelve Tasks of Asterix, The (Les douze travaux d'Astérix) (1976)"]

(26340, 7423)

In [419]:
index = 3869
print('rel: ', rel_all_items(tag)[index])
print('linear ',linear_sat(item,tag,direction)[index])
print('dimi ',diminish_sat(item,tag,direction)[index])
print('cos ',cos[index])
print('genre ',genre_simi[index])

rel:  0.682
linear  0.30287500000000006
dimi  0.780054392096477
cos  0.7926676759967288
genre  0.9128709291752769


In [420]:
index = 7423
print('rel: ', rel_all_items(tag)[index])
print('linear ',linear_sat(item,tag,direction)[index])
print('dimi ',diminish_sat(item,tag,direction)[index])
print('cos ',cos[index])
print('genre ',genre_simi[index])

rel:  0.8585
linear  0.39112500000000006
dimi  0.8585239734602474
cos  0.7824276446434084
genre  0.9128709291752769


In [407]:
rel_item(item,tag)

0.0762499999999999

In [410]:
rel_item(retrieved_item,tag)

0.682

In [405]:
sorted(score.items(), key=lambda x:x[1], reverse=True)[:30]

[("Twelve Tasks of Asterix, The (Les douze travaux d'Astérix) (1976)",
  0.6132054278393179),
 ('Toy Story (1995)', 0.6097923156447862),
 ('Monsters, Inc. (2001)', 0.5690709330005551),
 ('Finding Nemo (2003)', 0.5690085042875217),
 ('Shrek (2001)', 0.5644499150724894),
 ('Land Before Time, The (1988)', 0.5258969507800617),
 ('Rabbit Seasoning (1952)', 0.5237035308080275),
 ('Mickey, Donald, Goofy: The Three Musketeers (2004)', 0.5221382682016416),
 ('Toy Story 2 (1999)', 0.5210363773449698),
 ('101 Dalmatians (One Hundred and One Dalmatians) (1961)',
  0.5204273336982295),
 ('Robin Hood (1973)', 0.5174882737419338),
 ('Looney, Looney, Looney Bugs Bunny Movie, The (1981)', 0.5050147137371742),
 ('Who Framed Roger Rabbit? (1988)', 0.4964904965181962),
 ('Bugs Bunny / Road Runner Movie, The (a.k.a. The Great American Chase) (1979)',
  0.49541329158142033),
 ('Aladdin (1992)', 0.4940054467812984),
 ('Rudolph, the Red-Nosed Reindeer (1964)', 0.4930313943730364),
 ('Asterix and Cleopatra (As

## 5.2.3. Algorithm for Responding to Critiques

In [59]:
def sorting(item,critiques):
    for tag, direction in critiques:
        
    #critiques = [tag,direction]
    

IndentationError: expected an indented block after function definition on line 1 (2447653407.py, line 1)

In [152]:
item

'Shrek the Third (2007)'

In [156]:
critiques = [('classic',0.5),('007',-0.5)]

In [154]:
footballers_goals = {'Eusebio': 120, 'Cruyff': 104, 'Pele': 150, 'Ronaldo': 132, 'Messi': 125}

sorted_footballers_by_goals = sorted(footballers_goals.items(), key=lambda x:x[1], reverse=True)
print(sorted_footballers_by_goals)

[('Pele', 150), ('Ronaldo', 132), ('Messi', 125), ('Eusebio', 120), ('Cruyff', 104)]


In [153]:
sorted_footballers_by_goals[2]

('Messi', 125)

In [151]:
list(footballers_goals.items())[:4]

[('Eusebio', 120), ('Cruyff', 104), ('Pele', 150), ('Ronaldo', 132)]

In [70]:
footballers_goals.items()

dict_items([('Eusebio', 120), ('Cruyff', 104), ('Pele', 150), ('Ronaldo', 132), ('Messi', 125)])

In [64]:
a[0][1]

0.5